# Ejen Tempahan Hotel dan Penerbangan Contoh Sampel

Penyelesaian ini akan membantu anda menempah tiket penerbangan dan hotel. Senarionya adalah perjalanan dari London Heathrow LHR 20 Feb 2024 ke New York JFK balik pada 27 Feb 2025 terbang kelas ekonomi dengan British Airways sahaja. Saya mahu menginap di hotel Hilton di New York, sila berikan kos untuk penerbangan dan hotel.


# Inisialisasi Perkhidmatan Ejen AI Azure dan dapatkan maklumat konfigurasi dari **.env**

### **.env** 

Cipta fail .env 

**.env** mengandungi rentetan sambungan Perkhidmatan Ejen AI Azure, model yang digunakan oleh AOAI, dan perkhidmatan API Carian Google yang sepadan, ENDPOINT, dan lain-lain.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nama Penempatan Model Perkhidmatan Ejen AI Azure Anda"

[**NOTE**] Anda memerlukan model dengan Had Kadar 100,000 (Token per minit) Had Kadar 600 (Permintaan per minit)

  Anda boleh mendapatkan model di Microsoft Foundry - Model dan Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Rentetan Sambungan Projek Perkhidmatan Ejen AI Azure Anda"

  Anda boleh mendapatkan rentetan sambungan projek di gambaran keseluruhan projek anda dalam Skrin Portal AI ​​Foundry.

- **SERPAPI_SEARCH_API_KEY** = "KUNCI API Carian SERPAPI Anda"
- **SERPAPI_SEARCH_ENDPOINT** = "Endpoint Carian SERPAPI Anda"

Untuk mendapatkan Nama Penempatan Model dan Rentetan Sambungan Projek Perkhidmatan Ejen AI Azure, anda perlu mencipta Perkhidmatan Ejen AI Azure. Adalah disyorkan untuk menggunakan [templat ini](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) untuk menciptanya secara langsung （***Nota:*** Perkhidmatan Ejen AI Azure kini ditetapkan dalam wilayah terhad. Adalah disyorkan supaya anda merujuk kepada [pautan ini](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) untuk menetapkan wilayah)

Ejen perlu mengakses SERPAPI. Adalah disyorkan untuk mendaftar menggunakan [pautan ini](https://serpapi.com/searches). Selepas pendaftaran, anda boleh memperoleh KUNCI API unik dan ENDPOINT


# Setup 

Untuk menjalankan buku nota ini, anda perlu memastikan anda telah memasang perpustakaan yang diperlukan dengan menjalankan `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Versi Semantic Kernel anda sepatutnya sekurang-kurangnya 1.27.2.


Muatkan tetapan dan sumber fail .env anda, sila pastikan anda telah menambah kunci dan tetapan anda serta telah mencipta fail .env tempatan.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Log masuk ke Azure

Anda kini perlu log masuk ke Azure. Buka terminal dan jalankan arahan berikut:

```bash
az login
```

Arahan ini akan meminta anda untuk memasukkan kelayakan Azure anda, membolehkan perkhidmatan Ejen AI Azure berfungsi dengan betul.


# Explanation:
Ini adalah pembolehubah yang menyimpan kunci API untuk mengakses perkhidmatan API SERP (Search Engine Results Page). Kunci API adalah pengecam unik yang digunakan untuk mengesahkan permintaan yang dikaitkan dengan akaun anda.

Purpose: Tujuan baris ini adalah untuk menyimpan kunci API dalam pembolehubah supaya ia boleh digunakan untuk mengesahkan permintaan ke perkhidmatan API SERP. Kunci API diperlukan untuk mengakses perkhidmatan dan melakukan carian.
How to Get a SERP API Key: Untuk mendapatkan kunci API SERP, ikuti langkah umum ini di https://serpapi.com (langkah tepat mungkin berbeza bergantung pada perkhidmatan API SERP khusus yang anda gunakan):

Choose a SERP API Service: Terdapat beberapa perkhidmatan API SERP yang tersedia, seperti SerpAPI, Google Custom Search JSON API, dan lain-lain. Pilih yang paling sesuai dengan keperluan anda.

Sign Up for an Account: Lawati laman web perkhidmatan API SERP yang dipilih dan daftar untuk akaun. Anda mungkin perlu memberikan beberapa maklumat asas dan mengesahkan alamat e-mel anda.

Create an API Key: Selepas mendaftar, log masuk ke akaun anda dan navigasi ke bahagian API atau papan pemuka. Cari pilihan untuk mencipta atau menjana kunci API baru.
Copy the API Key to your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Ini adalah pembolehubah yang menyimpan URL asas untuk titik akhir API SERP. Nama pembolehubah BASE_URL adalah konvensyen yang digunakan untuk menunjukkan bahawa URL ini adalah titik permulaan untuk membuat permintaan API.
'https://serpapi.com/search':

Ini adalah rentetan URL sebenar yang diberikan kepada pembolehubah BASE_URL. Ia mewakili titik akhir untuk melakukan pertanyaan carian menggunakan API SERP.

# Purpose:
Tujuan baris ini adalah untuk mentakrifkan suatu pemalar yang menyimpan URL asas untuk API SERP. URL ini akan digunakan sebagai titik permulaan untuk membina permintaan API bagi melakukan operasi carian.

# Usage:
Dengan mentakrifkan URL asas dalam pembolehubah, anda boleh menggunakannya semula dengan mudah di seluruh kod anda bila-bila masa anda perlu membuat permintaan ke API SERP. Ini menjadikan kod anda lebih mudah diselenggara dan mengurangkan risiko kesilapan daripada menetapkan URL secara keras di pelbagai tempat. Contoh semasa adalah https://serpapi.com/search?engine=bing yang menggunakan API carian Bing. API yang berbeza boleh dipilih di https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Penjelasan:

Di sinilah kod plugin anda terletak.

Definisi Kelas: `class BookingPlugin`: Mendefinisikan kelas bernama BookingPlugin yang mengandungi kaedah untuk menempah hotel dan penerbangan.

Kaedah Tempahan Hotel:

- `@kernel_function(description="booking hotel")`: Hiasan yang menerangkan fungsi sebagai fungsi kernel untuk menempah hotel.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Mendefinisikan kaedah untuk menempah hotel dengan parameter dan jenis pulangan yang dianotasi.

Kaedah ini membina kamus parameter untuk permintaan tempahan hotel dan menghantar permintaan GET ke API SERP. Ia memeriksa status respons dan memulangkan hartanah hotel jika berjaya, atau None jika permintaan gagal.

Kaedah Tempahan Penerbangan:

- `@kernel_function(description="booking flight")`: Hiasan yang menerangkan fungsi sebagai fungsi kernel untuk menempah penerbangan.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Mendefinisikan kaedah untuk menempah penerbangan dengan parameter dan jenis pulangan yang dianotasi.

Kaedah ini membina kamus parameter untuk permintaan penerbangan keluar dan pulang dan menghantar permintaan GET ke API SERP. Ia memeriksa status respons dan menambah maklumat penerbangan ke rentetan hasil jika berjaya, atau mencetak mesej ralat jika permintaan gagal. Kaedah ini memulangkan rentetan hasil yang mengandungi maklumat penerbangan.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Penjelasan:
Pernyataan Import: Import modul yang diperlukan untuk kelayakan Azure, ejen AI, kandungan mesej sembang, peranan penulis, dan pemutus fungsi kernel.

Pengurus Konteks Asinkron: async dengan (DefaultAzureCredential() sebagai creds, AzureAIAgent.create_client(credential=creds, conn_str="...") sebagai client,): Ini menetapkan pengurus konteks asinkron untuk mengurus kelayakan Azure dan mencipta klien ejen AI.

Nama Ejen dan Arahan: 
- `AGENT_NAME = "BookingAgent"`: Menetapkan nama ejen.
- `AGENT_INSTRUCTIONS = """..."""`: Memberikan arahan terperinci untuk ejen mengenai cara mengendalikan permintaan tempahan.

Cipta Definisi Ejen: `agent_definition = await client.agents.create_agent(...)`: Mencipta definisi ejen dengan model, nama, dan arahan yang ditentukan.

Cipta Ejen AzureAI: `agent = AzureAIAgent(...)`: Mencipta ejen AzureAI menggunakan klien, definisi ejen, dan plugin yang ditentukan.

Cipta Thread: `thread: AzureAIAgentThread | None = None`: Cipta thread untuk ejen. Ia tidak wajib untuk mencipta thread terlebih dahulu - jika nilai `None` diberikan, satu thread baru akan dibuat semasa panggilan pertama dan dikembalikan sebagai sebahagian daripada respons.

Input Pengguna: `user_inputs = ["..."]`: Menetapkan senarai input pengguna untuk diproses oleh ejen.

Dalam blok finally, padam thread dan ejen untuk membersihkan sumber.


# Pengesahan

Kelas `DefaultAzureCredential` adalah sebahagian daripada Azure SDK untuk Python. Ia menyediakan cara lalai untuk mengesahkan dengan perkhidmatan Azure. Ia mencuba untuk mengesahkan menggunakan pelbagai kaedah dalam susunan tertentu, seperti pembolehubah persekitaran, identiti terurus, dan kelayakan Azure CLI.

Operasi Asinkronus: Modul aio menunjukkan bahawa kelas DefaultAzureCredential menyokong operasi asinkronus. Ini bermakna anda boleh menggunakannya dengan asyncio untuk melaksanakan permintaan pengesahan tanpa menyekat.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidakakuratan. Dokumen asal dalam bahasa asalnya hendaklah dianggap sebagai sumber yang sahih. Untuk maklumat penting, terjemahan profesional oleh manusia adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
